In [ ]:
rm -rf *.pyc

In [1]:
from CNN import data_set,reinitiate_set_params,build_update_functions
from sklearn.cross_validation import train_test_split
import theano
import theano.tensor as T
import numpy as  np
from faceKeySrc import shared_dataset
import time
import lasagne

Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN 5005)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
# path to train and testing data
PATH_train = "../data/training.csv"
PATH_test = "../data/test.csv"

### Pre-processing notes
    For zero centering data X -= np.mean(X,axis=0) if X [NxD] for N samples and D features
    For normalisation X/= np.std(X,axis=0) if X [NxD] for N samples of length D
    Also decorrelated, via PCA: Data has diagonal covariance matrix
    or Whitened: data has identity covariance matrix

### NOTE from cs231n: for images use centering only
    1. Subtract the mean image; for cifar10 subtract [32x32x3] as Alexnet
    2. or like VGG subtract per-channel , 3 numbers
    3. Apply whitening

In [3]:
data = data_set(path_train=PATH_train, path_test=PATH_test)

4909 samples from the total of 7049 have missing values
Missing values appear in 28 different columns of output targets
shape of X (7049, 9216) and y (7049, 30)


In [4]:
#  drop the missing values
data.drop_missing_values()
# center data VGG style
data.center_alexnet()

shape of X (2140, 9216) and y (2140, 30)
Training data has been centered alexnet style


In [5]:
# generate test validation split
train_set_x, valid_set_x, train_set_y, valid_set_y = train_test_split(
    data.X, data.y, test_size=0.2, random_state=42)

In [6]:
train_set_x = train_set_x.reshape(-1,1,96,96).astype(theano.config.floatX)
valid_set_x = valid_set_x.reshape(-1,1,96,96).astype(theano.config.floatX)
train_set_y = train_set_y.astype(theano.config.floatX)
valid_set_y = valid_set_y.astype(theano.config.floatX)


### small CNN

In [ ]:
# train the network
# parasms 
n_iter = 10000
improvement_threshold = 0.999
patience = 10000
max_fail = 10
#######################
batch_size = 128
n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size+1
n_valid_batches = valid_set_x.get_value(borrow=True).shape[0] // batch_size+1
patience_increase=.2
validation_frequency = min(n_train_batches, patience // 10)
best_val_loss_ = np.inf
train_loss_history_temp = []
########################
n_fail_ = 0
best_val_loss_ = np.inf
best_epoch_ = 0 
epoch = 0
done_looping = False
train_loss_history_ = []
val_loss_history_ =[]

In [ ]:
print 'HERE'

In [ ]:
start_time = time.time()
while (epoch < n_iter) and (not done_looping):
    epoch +=1
    
    # go over minibatches
    for minibatch_index in range(n_train_batches):
        
        # update network for one minibatch
        minibatch_average_cost,minibatch_average_RMSE = train_fn(minibatch_index)
        
        # store training loss of minibatches till the next validation step
        train_loss_history_temp.append(minibatch_average_RMSE)
        
        # number of minibatches checked
        num_minibatch_checked = (epoch -1) * n_train_batches + minibatch_index
        
        # if validation interval reached
        if (num_minibatch_checked+1 ) % validation_frequency ==0:
            
            # compute validation loss
            validation_losses = [val_fn(i)[0] for i in range(n_valid_batches)]
            
            # store mean validation loss for validation set
            current_val_loss = np.mean(validation_losses)
            
            # store training and validation history
            train_loss_history_.append(np.mean(train_loss_history_temp))
            val_loss_history_.append(current_val_loss)
            train_loss_history_temp = []

            # is it the best validation loss sofar?
            if current_val_loss < best_val_loss_ :
                
                # increase patience if improvement is significant
                if (current_val_loss < best_val_loss_*improvement_threshold):
                    patience = max(patience, num_minibatch_checked* patience_increase )
                
                # save the-sofar-best validation RMSE and epoch and model-params
                best_val_loss_ = current_val_loss
                best_epoch_ = epoch
                best_network_params = lasagne.layers.get_all_param_values(network)
                
        # check if patience exceeded and set the training loop to stop
        if (patience <= num_minibatch_checked):
            print 'patience reached'
            # reset the network weights to the best params saved
            print 'reseting the network params to that of the best seen'
            reinitiate_set_params(network=network,
                                  weights = best_network_params)
            # done optimising, break the optimisation loop
            done_looping = True
            break
   
    # print out results of optimising priodically
#     if num_minibatch_checked<1000:
#         freq = 200
#     elif num_minibatch_checked<5000:
#         freq = 500
#     elif num_minibatch_checked<10000:
#         freq = 1000
    freq = 10
        
    if (epoch % freq) == 0:
        print (('epoch %i, minibatch %i/%i, validation loss of %f, patience %i, in %f secs')%
               (epoch, minibatch_index+1, n_train_batches, current_val_loss, patience, time.time()-start_time ))
        start_time = time.time()
                
        

In [11]:
# print out variables in the work space
from types import ModuleType, FunctionType, TypeType
variables = %who_ls
Types_to_exclude = (ModuleType, FunctionType, TypeType)
variables_to_exclude = ('v','variables','Types_to_exclude','variables_to_exclude')
print '{:<15} {:<15}'.format('NAME', 'TYPE')
print '------------   ------------'
for v in variables:
    if not isinstance(eval(v), Types_to_exclude) and not str(v) in variables_to_exclude:
        print '{:<15} {:<15}'.format(v, type(eval(v)).__name__)


NAME            TYPE           
------------   ------------
PATH_test       str            
PATH_train      str            
X               TensorVariable 
data            data_set       
net             dict           
network         NonlinearityLayer
num_output      int            
train_set_x     ndarray        
train_set_y     ndarray        
valid_set_x     ndarray        
valid_set_y     ndarray        
y               TensorVariable 
